In [ ]:
# https://bigbuild.disruptions-app.com.au/api/disruptions/?ordering=start_date&search=Gippsland&type[]=18
# https://bigbuild.disruptions-app.com.au/api/disruptions/?ordering=start_date&search=Traralgon&type[]=18
# https://bigbuild.disruptions-app.com.au/api/disruptions/?ordering=start_date&search=Bairnsdale&type[]=18

# Type 18 is V/Line Disruptions

import datetime
from dateutil import parser
import requests
import icalendar
from pathlib import Path
import json
import re
import html2text

htmlConverter = html2text.HTML2Text()
htmlConverter.ignore_links = True
htmlConverter.ignore_emphasis = True

# A function that returns the 'year' value:
def myFunc(e):
  return e['start']

def cleanHtml(text):
    return re.sub(r"(?:(?!\n)\s)+", " ", 
                  htmlConverter.handle(text)
                  .replace("\\", '')
                  .replace("\n", ' ')
            ).strip()

def removeLinks(text):
    return re.sub(r"((For more information|Check the|visit the|--- Event details:|allow|See details|For further).*)|please", "", text,flags=re.IGNORECASE).strip() 

# on Victoria's Big Build visit https://bigbuild.vic.gov.au/disruptions", '').replace(r"For more information visit https://bigbuild.vic.gov.au/disruptions/disruptions-map",'')


# SUMMARY = name
# DTSTART = start_date
# DTEND = end_date
# X-ALT-DESC (DESCRIPTION) = description
# LOCATION = location


# updates = {}
datetimeformat = "%Y-%m-%d %H:%M:%S%z"
# datetimeformat = "%Y-%m-%d"
location = []
disruptionString = r'(\?|&)disruption=(<br>)?[\d]+'

In [23]:
updates = json.load(open(f'updates.json', "r"))


In [ ]:
fileNames = ['', '-type12']
names = ['gippsland', 'traralgon', 'bairnsdale']

for fileName in fileNames:
    for name in names:
        file = json.load(open(f'{name}{fileName}.json', "r"))
        for event in file:
            if str(event["id"]) in updates and "project" not in updates[str(update["id"])]:
                continue
            update = {}
            update["summary"] = event.get("name") #.replace('Gippsland Line - ', '').replace('Gippsland Line')
            update["location"] = event.get("location")
            update["project"] = int(re.search(r'[\d]+', event.get("project")).group())
            update["description"] = cleanHtml(event["description"]) # .replace("<p>", '').replace('</p>', '')
            update["start"] = parser.parse(event["start_date"]).strftime(datetimeformat)
            update["end"] = parser.parse(event["end_date"]).strftime(datetimeformat)
            update["id"] = event["id"]
            update["description"] = removeLinks(update["description"])
            
            if str(event["id"]) in updates:
                print("found" + str(event["id"]))
                existUpdate = updates[str(update["id"])]
                existUpdate["project"] = update["project"] 
                if len(existUpdate["summary"]) < len(update["summary"]):
                    existUpdate["summary"] = update["summary"]
                if len(existUpdate["description"]) < len(update["description"]):
                    existUpdate["description"] = update["description"]
            else:
                print("creating" + str(event["id"]))
                updates[str(update["id"])] = update
            
    # updates.append(update)
    # else:
    #     print(update)
    #     i = i+1


# len(updates)
# updates.sort(key=myFunc)

updates
# location

In [ ]:
i = 1
ics_path = Path('disruption-calendar.ics')
calendar = icalendar.Calendar.from_ical(ics_path.read_bytes())
for event in calendar.events:
    update = {}
    update["summary"] = str(event.get("SUMMARY"))
    update["location"] = str(event.get("LOCATION", None))
    description = event["X-ALT-DESC"]
    update["description"] = removeLinks(cleanHtml(description))
    update["start"] = event.DTSTART.strftime(datetimeformat)
    update["end"] = event.DTEND.strftime(datetimeformat)
    id = re.search(disruptionString, description)

    if id is not None: 
        update["id"] = int(re.search(r'[\d]+', id.group()).group())
    else:
        addevent = re.search(r'https://www.addevent.com/event/[0-9A-z]+', description)
        if addevent is not None:
            event = requests.get(addevent.group()).text
            id = re.search(disruptionString, event)
    
            # Not worth it. All the Add Event Disruptions are well documented, and we'd need to clean this up.
            # update["description"] = cleanHtml(event)
            # print(update["description"])

            if id is not None:
                update["id"] = int(re.search(r'[\d]+', id.group()).group())
            else:
                # print(event)
                # print(addevent.group())
                # print(addevent.group())
                update["id"] = 'Unknowndv' + str(i)
                i = i + 1
        else:
            update["id"] = 'Unknown' + str(i)
            print(description)
            i = i + 1

    if update["id"] in updates:
        # print(update["id"])
        existUpdate = updates[update["id"]]
        if len(existUpdate["summary"]) < len(update["summary"]):
            existUpdate["summary"] = update["summary"]
        if len(existUpdate["description"]) < len(update["description"]):
            existUpdate["description"] = update["description"]
    else:
        updates[update["id"]] = update


In [ ]:
len(updates)

In [26]:
json_output = json.dumps(updates)
with open(f'updates.json', "w") as f:
    f.write(json_output)

In [5]:
projects = {}

for key, update in updates.items():
    if "project" in update:
        if update["project"] not in projects:
            projects[update["project"]] = 0
        
        projects[update["project"]] = projects[update["project"]] + 1
projects
    